In [ ]:
import types
import paramiko
import time
import random
import string
import sys


def random_interface_name(prefix="tuntap", length=5):
    """生成随机接口名称"""
    suffix = "".join(
        random.choice(string.ascii_lowercase + string.digits) for _ in range(length)
    )
    return f"{prefix}.{suffix}"


def create_and_remove_tuntap(ssh_client, interface_type="tun", delay=2):
    """创建、设置为up并删除一个TUN/TAP接口"""
    interface_name = random_interface_name()

    try:
        # 创建TUN/TAP接口
        print(f"创建 {interface_type} 接口: {interface_name}")
        stdin, stdout, stderr = ssh_client.exec_command(
            f" ip tuntap add dev {interface_name} mode {interface_type}"
        )
        exit_status = stdout.channel.recv_exit_status()
        if exit_status != 0:
            print(f"创建接口失败: {stderr.read().decode('utf-8').strip()}")
            return False

        # 设置接口为UP状态
        print(f"启用接口: {interface_name}")
        stdin, stdout, stderr = ssh_client.exec_command(
            f" ip link set {interface_name} up"
        )
        exit_status = stdout.channel.recv_exit_status()
        if exit_status != 0:
            print(f"启用接口失败: {stderr.read().decode('utf-8').strip()}")

        # 等待指定的延迟时间
        time.sleep(delay)

        # 删除接口
        print(f"删除接口: {interface_name}")
        stdin, stdout, stderr = ssh_client.exec_command(
            f"sudo ip tuntap del dev {interface_name} mode {interface_type}"
        )
        exit_status = stdout.channel.recv_exit_status()
        if exit_status != 0:
            print(f"删除接口失败: {stderr.read().decode('utf-8').strip()}")
            return False

        return True

    except Exception as e:
        print(f"操作失败: {str(e)}")
        return False


def get_user_input():
    print("SSH连接并配置随机TUN/TAP接口")

    # 收集用户输入
    host = input("SSH主机地址 (格式: user@hostname): ")
    if not host.strip():
        host = "root@192.168.1.189"

    # port_input = input("SSH端口 (默认: 22): ")
    # port = int(port_input) if port_input.strip() else 22
    port = 22

    interface_type = input("接口类型 (tun/tap) (默认: tun): ").lower()
    if interface_type not in ["tun", "tap"]:
        interface_type = "tun"

    delay_input = input("每次操作的间隔时间(秒) (默认: 2): ")
    delay = float(delay_input) if delay_input.strip() else 2.0

    iterations_input = input("操作次数 (默认: 2): ")
    iterations = int(iterations_input) if iterations_input.strip() else 2

    key = input("SSH私钥文件路径 (可选，按Enter跳过): ")
    if not key.strip():
        key = None

    password = None
    if not key:
        password = input("SSH密码 (不推荐，优先使用密钥认证) (可选，按Enter跳过): ")
        if not password.strip():
            password = ""

    # 返回参数class
    return types.SimpleNamespace(
        host=host,
        port=port,
        type=interface_type,
        delay=delay,
        iterations=iterations,
        key=key,
        password=password,
    )


def main():
    args = get_user_input()

    # 初始化SSH客户端
    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        # 建立SSH连接
        print(f"连接到 {args.host}:{args.port}...")

        # 确定认证方式
        if args.key:
            ssh_client.connect(
                hostname=args.host.split("@")[1] if "@" in args.host else args.host,
                username=args.host.split("@")[0] if "@" in args.host else None,
                port=args.port,
                key_filename=args.key,
            )
        else:
            ssh_client.connect(
                hostname=args.host.split("@")[1] if "@" in args.host else args.host,
                username=args.host.split("@")[0] if "@" in args.host else None,
                port=args.port,
                password=args.password,
            )

        print("连接成功！")

        # 检查sudo权限
        print("检查sudo权限...")
        stdin, stdout, stderr = ssh_client.exec_command("sudo -n true")
        if stdout.channel.recv_exit_status() != 0:
            print("警告: 可能需要输入sudo密码。建议配置无密码sudo访问以避免脚本中断。")

        # 执行TUN/TAP接口操作
        count = 0
        try:
            while args.iterations == 0 or count < args.iterations:
                if create_and_remove_tuntap(ssh_client, args.type, args.delay):
                    count += 1
                    print(
                        f"完成操作 {count}"
                        + (
                            " (无限循环)"
                            if args.iterations == 0
                            else f"/{args.iterations}"
                        )
                    )
                else:
                    print("操作失败，重试...")

                # 短暂暂停
                time.sleep(1)

        except KeyboardInterrupt:
            print("\n用户中断操作。")

    except paramiko.AuthenticationException:
        print("认证失败。请检查用户名、密码或密钥。")
        sys.exit(1)
    except paramiko.SSHException as e:
        print(f"SSH连接错误: {str(e)}")
        sys.exit(1)
    except Exception as e:
        print(f"发生错误: {str(e)}")
        sys.exit(1)
    finally:
        ssh_client.close()
        print("连接已关闭。")


if __name__ == "__main__":
    main()

SSH连接并配置随机TUN/TAP接口
连接到 root@192.168.1.189:22...
连接成功！
检查sudo权限...
创建 tun 接口: tuntap.8sycb
启用接口: tuntap.8sycb
删除接口: tuntap.8sycb
完成操作 1/2
创建 tun 接口: tuntap.b8rxn
启用接口: tuntap.b8rxn
删除接口: tuntap.b8rxn
完成操作 2/2
连接已关闭。
